In [1]:
import os
import pandas as pd
import numpy as np
import datetime
state_dict = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ',
    'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 
    'Connecticut': 'CT', 'District Of Columbia':"DC", 'Delaware': 'DE', 'Florida': 'FL', 
    'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL',
    'Indiana': 'IN', 'Iowa': 'IA','Kansas': 'KS', 'Kentucky': 'KY',
    'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI',
    'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX',
    'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', "United States": "US", "United States Total": "US"}


In [2]:
def is_substring_in_array(substring, array):
    # Use np.char.find to find the substring in each element of the array
    found = np.char.find(array, substring)
    # Check if any position is not -1
    return np.any(found != -1)

In [3]:

dfs_dct = {}
ab_keys = {"a": "United States", "b": "Missouri"}
start_year = 1998
end_year = 2022
folder = "StateFinancesByYear"
null_years = [1997, 2001, 2003, 2004]
for year in range(start_year,2022):
    if year in null_years:
        continue
    dfs_dct[year] = {}        
    stryear = str(year)
    filetype = ".xlsx" if year > 2011 else ".xls"

    for ab in ["a", "b"]:
        filename = f"{folder}/StateGovFinances{stryear}{ab}{filetype}"
        try:
            df = pd.read_excel(filename)#dropna(axis = 0, how = "any")
        except:
            print(stryear+ab, "failed")
            break
        index = [key for key in df if "Table" in key]
        if len(index) != 0:
            df = df.set_index(index)#.iloc[1:]
        else:
            df = df.set_index(df.columns[0])
        df.index = ([str(ind).split('(')[0].strip() for ind in df.index])

        if year  < 1998:
            df[list(df.keys())[:]]
            df = df[list(df.keys())[4:]]
        # try:
        
        for row in range(df.shape[0]):
            in_array = is_substring_in_array(ab_keys[ab].title(), str(df.iloc[row]).title())
            if in_array:
                break #if is_string_array(df.iloc[row].values) else True:
            else:
                continue
        # print(year, row)
        # except:
        #     while ab_keys[ab] not in df.iloc[row].values:
        #         row +=1     
        #       
        df.iloc[row] = df.iloc[row].str.title() 
        df.iloc[row] = df.iloc[row].fillna(method='ffill')
        df.iloc[row + 1] = df.iloc[row + 1].str.strip() + " " + df.iloc[row +2].str.replace("1", "").str.strip()+ " " + df.iloc[row +3].str.replace("1", "").str.strip()
        index = list(df.index)
        index[row+1] = "Variable"
        # df.set_index(index, inplace = True)
        # df.set_index(list(df.keys())[row, row + 1]) 
        # df.columns = df.iloc[[row, row + 1]].T

        # df = df.iloc[row + 6:].dropna(how = "all")
        df = df.iloc[row:].dropna(how = "all")

        df.index = [str(ix).strip().replace("1", "").upper() for ix in df.index]
        df_index_count = df.index.to_series().groupby(df.index).cumcount().add(1).astype(str)
        df.index = df_index_count + df_index_count.index
        tdf = df.dropna(how = "all").T
        cols = list(tdf.iloc[1:].columns)
        cols[0] = "State"
        cols[1] = "Format"
        tdf.columns = cols
        tdf = tdf.iloc[1:]
        df = tdf    
        df["State"] = df["State"].apply(lambda x: state_dict[x.strip()])     
        df["Year"] = datetime.datetime(year,1,1)
        df.set_index(["State", "Year", "Format"], inplace = True)
        keys = list(df.keys())[4:]
        
        dfs_dct[year][ab] = df[keys]#reset_index().set_index(["Year", "index"])#.T
for year, ab_dct in dfs_dct.items():
    if "b" in ab_dct.keys():
        # dfs_dct[year] = pd.merge(ab_dct["a"], ab_dct["b"], how = "outer", left_index = True, right_index = True)
        dfs_dct[year] = pd.concat([ab_dct["a"], ab_dct["b"]])        
    else:
        dfs_dct[year] = ab_dct["a"]
    dfs_dct[year] = dfs_dct[year]
    # dfs_dct[year].set_index("index", inplace = True)
    # dfs_dct[year].index.name = "Description"

/tmp/ipykernel_17686/3717898496.py:45: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.iloc[row] = df.iloc[row].fillna(method='ffill')
/tmp/ipykernel_17686/3717898496.py:45: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.iloc[row] = df.iloc[row].fillna(method='ffill')
/tmp/ipykernel_17686/3717898496.py:45: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.iloc[row] = df.iloc[row].fillna(method='ffill')
/tmp/ipykernel_17686/3717898496.py:45: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.iloc[row] = df.iloc[row].fillna(method='ffill')
/tmp/ipykernel_17686/3717898496.py:45: FutureWarning: Series.fillna with 'method' is deprecated and 

2017b failed
2018b failed


/tmp/ipykernel_17686/3717898496.py:45: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.iloc[row] = df.iloc[row].fillna(method='ffill')
/tmp/ipykernel_17686/3717898496.py:45: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.iloc[row] = df.iloc[row].fillna(method='ffill')


2019b failed


/tmp/ipykernel_17686/3717898496.py:45: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.iloc[row] = df.iloc[row].fillna(method='ffill')
/tmp/ipykernel_17686/3717898496.py:45: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.iloc[row] = df.iloc[row].fillna(method='ffill')


2020b failed
2021b failed


In [4]:
stack_dfs = pd.concat([dfs_dct[year] for year in dfs_dct.keys()])#.reset_index().set_index(["index", "Year"])
stack_dfs

1PERSONAL INCOME    1REVENUE  \
State Year       Format                                                         
US    1998-01-01 State & local government CV                  (X)        0.06   
                 State government amount                  6909681  1103239093   
                 Local government amount                  6928762   909661010   
                 Local government CV                          (X)        0.11   
AL    1998-01-01 State & local government amount            91283    22400024   
...                                                           ...         ...   
WY    2021-01-01 State & local government amount              NaN    11886452   
                 State & local government CV                  NaN        0.18   
                 State government amount                      NaN     8179668   
                 Local government amount                      NaN     5556067   
                 Local government CV                          NaN        0.44   

                                                 1GENERAL REVENUE  \
State Year       Format                                             
US    1998-01-01 State & local government CV                 0.07   
                 State government amount                863522535   
                 Local government amount                794250164   
                 Local government CV                         0.13   
AL    1998-01-01 State & local government amount         18232720   
...                                                           ...   
WY    2021-01-01 State & local government amount          9844705   
                 State & local government CV                 0.19   
                 State government amount                  6375373   
                 Local government amount                  5318614   
                 Local government CV                         0.41   

                                                 1INTERGOVERNMENTAL REVENUE  \
State Year       Format                                                       
US    1998-01-01 State & local government CV                           0.08   
                 State government amount                          240788817   
                 Local government amount                          306270014   
                 Local government CV                                   0.09   
AL    1998-01-01 State & local government amount                    4330773   
...                                                                     ...   
WY    2021-01-01 State & local government amount                    3433945   
                 State & local government CV                           0.36   
                 State government amount                            3103070   
                 Local government amount                            2180157   
                 Local government CV                                   0.78   

                                                 1FROM FEDERAL GOVERNMENT  \
State Year       Format                                                     
US    1998-01-01 State & local government CV                         0.08   
                 State government amount                        224443723   
                 Local government amount                         30604373   
                 Local government CV                                 0.66   
AL    1998-01-01 State & local government amount                  4330773   
...                                                                   ...   
WY    2021-01-01 State & local government amount                  3433945   
                 State & local government CV                         0.36   
                 State government amount                          3089280   
                 Local government amount                           344665   
                 Local government CV                                 3.55   

                                                 1FROM STATE GOVERN

In [5]:
panel_dfs_dict = {k:stack_dfs[stack_dfs.index.get_level_values(2)==k].reset_index().set_index(["State","Year"]).sort_index() for k in stack_dfs.index.get_level_values(2).unique()}
panel_dfs_dict['State & local government amount'][["1GENERAL REVENUE","1EXPENDITURE"]]

1GENERAL REVENUE 1EXPENDITURE
State Year                                    
AK    1998-01-01          9662079      7753932
      1999-01-01          7611678      8085378
      2000-01-01          9042205      8627637
      2002-01-01          7208865      9397434
      2005-01-01          9824780     10026883
...                           ...          ...
WY    2017-01-01          8728100     10357605
      2018-01-01          8668777     10129438
      2019-01-01          8771293      9986772
      2020-01-01         10018385     10440183
      2021-01-01          9844705     11386915

[1071 rows x 2 columns]

In [6]:
print(list(stack_dfs.keys()))
stack_dfs[["1GENERAL REVENUE","1EXPENDITURE", '1INDIVIDUAL INCOME', '1CORPORATE INCOME']]

['1PERSONAL INCOME', '1REVENUE', '1GENERAL REVENUE', '1INTERGOVERNMENTAL REVENUE', '1FROM FEDERAL GOVERNMENT', '1FROM STATE GOVERNMENT', '1FROM LOCAL GOVERNMENTS', '1GENERAL REVENUE FROM OWN SOURCES', '1TAXES', '1PROPERTY', '1SALES AND GROSS RECEIPTS', '1GENERAL SALES', '1SELECTIVE SALES', '1MOTOR FUEL', '1ALCOHOLIC BEVERAGE', '1TOBACCO PRODUCTS', '1PUBLIC UTILITIES', '1OTHER SELECTIVE SALES', '1INDIVIDUAL INCOME', '1CORPORATE INCOME', '1MOTOR VEHICLE LICENSE', '1OTHER TAXES', '1CHARGES AND MISCELLANEOUS GENERAL  REVENUE', '1CURRENT CHARGES', '1EDUCATION', '1INSTITUTIONS  OF HIGHER EDUCATION', '1SCHOOL LUNCH SALES', '1HOSPITALS', '1HIGHWAYS', '1AIR TRANSPORTATION', '1PARKING FACILITIES', '1SEA AND INLAND PORT FACILITIES', '1NATURAL RESOURCES', '1PARKS AND RECREATION', '1HOUSING AND COMMUNITY DEVELOPMENT', '1SEWERAGE', '1SOLID WASTE MANAGEMENT', '1OTHER CHARGES', '1MISCELLANEOUS GENERAL REVENUE', '1INTEREST EARNINGS', '1SPECIAL ASSESSMENTS', '1SALE OF PROPERTY', '1OTHER GENERAL REVENUE'

1GENERAL REVENUE  \
State Year       Format                                             
US    1998-01-01 State & local government CV                 0.07   
                 State government amount                863522535   
                 Local government amount                794250164   
                 Local government CV                         0.13   
AL    1998-01-01 State & local government amount         18232720   
...                                                           ...   
WY    2021-01-01 State & local government amount          9844705   
                 State & local government CV                 0.19   
                 State government amount                  6375373   
                 Local government amount                  5318614   
                 Local government CV                         0.41   

                                                 1EXPENDITURE  \
State Year       Format                                         
US    1998-01-01 State & local government CV             0.06   
                 State government amount            929951588   
                 Local government amount            884758684   
                 Local government CV                      0.1   
AL    1998-01-01 State & local government amount     21652215   
...                                                       ...   
WY    2021-01-01 State & local government amount     11386915   
                 State & local government CV             0.12   
                 State government amount              7494943   
                 Local government amount              5386454   
                 Local government CV                     0.26   

                                                 1INDIVIDUAL INCOME  \
State Year       Format                                               
US    1998-01-01 State & local government CV                   0.03   
                 State government amount                  160114849   
                 Local government amount                   15515186   
                 Local government CV                           0.32   
AL    1998-01-01 State & local government amount            1887166   
...                                                             ...   
WY    2021-01-01 State & local government amount                  0   
                 State & local government CV                      0   
                 State government amount                          0   
                 Local government amount                          0   
                 Local government CV                              0   

                                                 1CORPORATE INCOME  
State Year       Format                                             
US    1998-01-01 State & local government CV                     0  
                 State government amount                  31088625  
                 Local government amount                   3322990  
                 Local government CV                             0  
AL    1998-01-01 State & local government amount            243803  
...                                                            ...  
WY    2021-01-01 State & local government amount                 0  
                 State & local government CV                     0  
                 State government amount                         0  
                 Local government amount                         0  
                 Local government CV                             0  

[5126 rows x 4 columns]

In [7]:
before2009 = pd.read_csv("StateGovFinancesTo2008.csv", parse_dates=True)
before2009["Year4"] = pd.to_datetime(before2009["Year4"], format = "%Y")
before2009["State"] = before2009["Name"].str[:2]
before2009 = before2009.set_index(["State","Year4"]).sort_index()
names = ["Population (000)", "Personal Income ($mil)", "Total Revenue", "Total Rev-Own Sources",	
         "Total Taxes", "Property Tax (T01)", "Special Assessments (U01)", "Total IG Revenue", "Total Fed IG Revenue"]
before2009[names]

Population (000)  Personal Income ($mil)  Total Revenue  \
State Year4                                                                 
AK    1957-01-01             231.0                     539          38652   
      1958-01-01             224.0                     559          46252   
      1959-01-01             224.0                     593          50024   
      1960-01-01             229.0                     701          74646   
      1961-01-01             238.0                     698          79609   
...                            ...                     ...            ...   
WY    2004-01-01             503.0                   18239        4609487   
      2005-01-01             507.0                   18902        5296649   
      2006-01-01             513.0                   22912        5831362   
      2007-01-01             523.0                   24457        6091326   
      2008-01-01             533.0                   25892        6718235   

                  Total Rev-Own Sources  Total Taxes  Property Tax (T01)  \
State Year4                                                                
AK    1957-01-01                  27789        20974                   2   
      1958-01-01                  31306        23263                   1   
      1959-01-01                  31687        24045                  32   
      1960-01-01                  41305        27110                 542   
      1961-01-01                  48585        31571                 219   
...                                 ...          ...                 ...   
WY    2004-01-01                3175375      1504777              139809   
      2005-01-01                3361770      1739646              180821   
      2006-01-01                3822058      2122239              217907   
      2007-01-01                4144068      2025090              266498   
      2008-01-01                4553583      2404843              278812   

                  Special Assessments (U01)  Total IG Revenue  \
State Year4                                                     
AK    1957-01-01                          0             10863   
      1958-01-01                          0             14946   
      1959-01-01                          0             18337   
      1960-01-01                          0             33341   
      1961-01-01                          0             31024   
...                                     ...               ...   
WY    2004-01-01                          0           1434112   
      2005-01-01                          0           1934879   
      2006-01-01                          0           2009304   
      2007-01-01                          0           1947258   
      2008-01-01                          0           2164652   

                  Total Fed IG Revenue  
State Year4                             
AK    1957-01-01                 10713  
      1958-01-01                 14786  
      1959-01-01                 18191  
      1960-01-01                 33081  
      1961-01-01                 30974  
...                                ...  
WY    2004-01-01               1369382  
      2005-01-01               1824032  
      2006-01-01               1957575  
      2007-01-01               1776003  
      2008-01-01               1997454  

[3259 rows x 9 columns]

In [8]:
before2009["Special Assessments (U01)"]

State  Year4     
AK     1957-01-01    0
       1958-01-01    0
       1959-01-01    0
       1960-01-01    0
       1961-01-01    0
                    ..
WY     2004-01-01    0
       2005-01-01    0
       2006-01-01    0
       2007-01-01    0
       2008-01-01    0
Name: Special Assessments (U01), Length: 3259, dtype: int64

In [9]:
stack_dfs.to_csv(f"StateGovFinances{start_year}to2021.csv")

In [10]:
stack_dfs.index.get_level_values("Format").unique() 


Index(['State & local government CV', 'State government amount',
       'Local government amount', 'Local government CV',
       'State & local government amount'],
      dtype='object', name='Format')

In [11]:
stack_dfs[stack_dfs.index.get_level_values("Format") == "State & local government amount"]

,,,1PERSONAL INCOME,1REVENUE,1GENERAL REVENUE,1INTERGOVERNMENTAL REVENUE,1FROM FEDERAL GOVERNMENT,1FROM STATE GOVERNMENT,1FROM LOCAL GOVERNMENTS,1GENERAL REVENUE FROM OWN SOURCES,1TAXES,1PROPERTY,...,5NAN,6NAN,1FROM LOCAL GOVERNMENT,1LICENSE TAXES,1INSTITUTIONS OF HIGHER EDUCATION,1OTHER THAN GENERAL REVENUE,1SOCIAL SERVICES AND INCOME MAINTENANCE:,1EMPLOYMENT SECURITY ADMINISTRATION,1INSURANCE TRUST REVENUE2,1CHARGES AND MISCELLANEOUS GENERAL REVENUE
State,Year,Format,,,,,,,,,,,,,,,,,,,,,
AL,1998-01-01,State & local government amount,91283,22400024,18232720,4330773,4330773,(1),(1),13901947,8336957,1083600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AK,1998-01-01,State & local government amount,16465,11021929,9662079,1250101,1250101,(1),(1),8411978,2013461,705791,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZ,1998-01-01,State & local government amount,103704,26119257,18716134,3533902,3533902,(1),(1),15182232,11072454,3166045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AR,1998-01-01,State & local government amount,51059,12759027,10465148,2468930,2468930,(1),(1),7996218,5438434,903589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CA,1998-01-01,State & local government amount,862114,232301900,174426572,32551849,32551849,(1),(1),141874723,98716033,24722035,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VA,2021-01-01,State & local government amount,NaN,135480993,100596796,24701904,24701904,0,0,75894892,53564770,16566844,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147576,31769162,22330122
WA,2021-01-01,State & local government amount,NaN,148243176,100010400,24457914,24457914,0,0,75552486,51574328,14711643,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,294913,40266301,23978157
WV,2021-01-01,State & local government amount,NaN,26944612,20271833,7283083,7283083,0,0,12988750,8386049,1921893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32021,6226908,4602700
